## 1. Data manipulate

The goal is to add a `transcript` column into overlap res. Each `transcript` has a unique `tss`. 

- use [foverlap](https://www.rdocumentation.org/packages/data.table/versions/1.11.8/topics/foverlaps) function to get `peak` overlapping with `TSS` regions.  

### 1.1 load all transcripts

In [2]:
require(data.table)
#require(rtracklayer)
#all.transcript <- import.bed('../dat/1901/gencode.v19.1kb_all_possible_transcripts.bed')
all.transcript <- fread('../dat/1901/gencode.v19.1kb_all_possible_transcripts.bed',
                       col.names=c("seq",'start','end','gene'))
setkey(all.transcript,seq,start,end) # this step changed order 
head(all.transcript)
tail(all.transcript)
dim(all.transcript)

seq,start,end,gene
chr1,68591,69591,OR4F5
chr1,138879,139879,AL627309.1
chr1,367140,368140,OR4F29
chr1,621553,622553,OR4F16
chr1,738637,739637,AL669831.1
chr1,817543,818543,AL645608.2


seq,start,end,gene
chrY,59213544,59214544,VAMP7
chrY,59213547,59214547,VAMP7
chrY,59329752,59330752,IL9R
chrY,59329877,59330877,IL9R
chrY,59329905,59330905,IL9R
chrY,59338285,59339285,IL9R


[1] 133870      4

In [2]:
all.transcript[133453,]
all.transcript[42733,]

seq,start,end,gene
chrX,153718453,153719453,SLC10A3


seq,start,end,gene
chr15,67546492,67547492,AAGAB


In [5]:
saveRDS(all.transcript,'../dat/1901/gencode.v19.1kb_all_possible_transcripts.Rds')

### fread changed order of rows (reordered based on seq) !!!
- because set key 

## 3. Repeated for detla

In [4]:
require(tidyverse)

In [5]:
overlap.res <- fread('../dat/1901/delta.promoter.long_matrix.txt')%>%
    separate(peak,into = c("seq","start","end"),sep = ":|-")%>%
    mutate(seq=paste0('chr',seq))
setDT(overlap.res)
overlap.res <-overlap.res[,c("start","end"):=lapply(.SD, as.numeric), .SDcols = c("start","end")]


overlap.res <- overlap.res%>%
    mutate(transcript.idx=foverlaps(x = overlap.res%>%select(-gene),y = all.transcript,
                 by.x=c("seq", "start", "end"),
                 type="any", which=T,mult="first"))%>%
    select(-overlap)

##
head(overlap.res)
dim(overlap.res)

seq,start,end,gene,cluster,barcode,transcript.idx
chr1,859052,860562,SAMD11,delta_1,Islet3-fresh_CGGCTATGGCTCATGACTAAGCCTGTACTGAC,7
chr1,859052,860562,SAMD11,delta_1,Islet3-fresh_AGACACCTACTCGCTATCTCTCCGGTACTGAC,7
chr1,859052,860562,SAMD11,delta_1,Islet3-fresh_AGACACCTACTCGCTATCTCTCCGGTACTGAC,7
chr1,859052,860562,SAMD11,delta_1,Islet2-fresh_TCGAGGCACGTACTAGTCTAGCTAGGC,7
chr1,859052,860562,SAMD11,delta_1,Islet3-fresh_AGCGATAGTAGGCATGTTATGCGATATAGCCT,7
chr1,859052,860562,SAMD11,delta_1,Islet3-fresh_CGGCTATGGCTCATGACTAAGCCTGTACTGAC,7


[1] 2330095       7

In [16]:
range(overlap.res$transcript.idx)
dim(overlap.res)

[1]      7 133795

[1] 2330095       7

In [17]:
all.transcript[7,]

seq,start,end,gene
chr1,859760,860760,SAMD11


##  save

In [18]:
fwrite(overlap.res,'../dat/1901/delta.promoter.long_matrix_w_transcripts_corrected.txt')

### check idx

`(base) ➜  1901 git:(master) ✗ grep ABHD14A alpha_beta.promoter.long_matrix_w_transcripts_corrected.txt |awk -v FS=',' '{print $1,$2,$3,$4,$7}'| sort|uniq`

chr3 52007627 52009641 ABHD14A 92628

chr3 52007627 52009641 ABHD14A-ACY1 92628

`(base) ➜  1901 git:(master) ✗ grep ABHD14A alpha_beta.promoter.long_matrix_w_transcripts.txt |awk -v FS=',' '{print $1,$2,$3,$4,$7}'| sort|uniq`

chr3 52007627 52009641 ABHD14A 92628

chr3 52007627 52009641 ABHD14A-ACY1 92628

In [13]:
all.transcript[92628,]

seq,start,end,gene
chr3,52007200,52008200,ABHD14A
